# Integer Management

Dieses Notebook beschreibt alle Funktionen, die zur Speicherung und Umwandlung der `chess.Board`-Objekte in Integerwerte benötigt werden.
Gleichermaßen wird auch eine Funktion definiert, die aus einem Integer ein `chess.Board` erstellt.

Die Funktion ``to_integer(board, piece_list)`` berechnet für ein ``board`` und eine Liste der darauf befindlichen Figuren ``piece_list`` diese Zahl. Es ist wichtig die Liste der Figuren als Übergabeparameter zu erhalten und nicht zu berechnen, da die Position einer Figur in der Liste mit der Position im Binärcode übereinstimmt. Bei der Dekodierung wird aus dieser Position die Figur bestimmt. Das geringwertigste Bit speichert die Information, welcher Spieler am Zug ist. Die folgenden (pro Figur 7) Bits kodieren die Zahl, welche der Position einer Figur auf dem Spielfeld entspricht. Die Reihenfolge der Figuren entspricht deren Reihenfolge in der ``piece_list``. Hierbei gilt es zu berücksichtigen, dass die Feldpositionen mit dem Wert `chess.Square`+1 abgespeichert werden. Dies hat den Hintergrund, da das erste Feld der `chess` Bibliothek mit 0 referenziert wird, was bei einer Dekodierung nicht von einem leeren Integer unterschieden werden kann.

Die Funktion gibt einen Integer mit 24 oder 32 Byte Größe (32 Byte bei Bauern in der Stellung) zurück, welcher die Spielsituation repräsentiert.

Beispiel der Kodierung:
Für das Board:
```
. . . . k . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . Q . .
. . . . K . . .
```
Mit der zugehörigen piece_list: ``[chess.Piece.from_symbol('k'), chess.Piece.from_symbol('K'), chess.Piece.from_symbol('Q')]`` wird zuerst die ``piece_map`` bestimmt: ``{4: Piece.from_symbol('K'), 60: Piece.from_symbol('k'), 13: Piece.from_symbol('Q')}`` (Hinweis: Für dieses Beispiel wurde die Reihenfolge der Elemente in der ``piece_map`` vertauscht)

Anschließend wird der Spieler am Zug in das erste Bit kodiert. Dieses ist somit ``0`` oder ``1``.

Zum Kodieren der Positionen wird über die Inhalte der Piece Map iteriert. Das erste Element ist der schwarze König ``k``. Für dieses wird der Index in der ``piece_list`` bestimmt. Dieser ist 0. Die Position 60 (aus der ``piece_map``) wird nun an die Positionen 1 bis 7 (Index 0 in ``piece_list``) der Binärzahl zu ``0111101`` kodiert (Zur Erinnerung: Abspeicherung = `chess.Square`+ 1). Für das nächste Element (``K``) wird die Position (4) an die Stellen 8 bis 14 (Index 1 in ``piece_list``) zu ``0000101`` kodiert. Für das letzt Element (``Q``) wird die Position (13) an die Stellen 15 bis 21 (Index 2 in ``piece_list``) zu ``0001110`` kodiert.

Daraus entsteht die folgende Binärzahl (sofern Weiß am Zug ist): ``0001110000010101111011``, welche von der Funktion als Integer ``460155`` zurückgegeben wird.

Ein Sonderfall liegt in Situationen mit Bauern vor.
Da diese eine Änderung der Figuren in der Stellung enthalten, müssen die möglichen Figuren im Integer beachtet werden.
Wenn ein Bauer in der Situation ist, kann keine Dame vorhanden sein.
Daher wird diese bei der Kodierung mit der Position 127 (126 in der piece_map) gespeichert und bei der Dekodierung herausgefiltert.
Aus diesem Grund müssen auch für alle anderen Positionen 7 Bit in Anspruch genommen werden.



In [ ]:
def to_integer(board, piece_list):
    
    tmp_list = piece_list.copy()
    piece_map = board.piece_map()
    int_rep = int(board.turn)

    pawn = chess.Piece.from_symbol("P")
    if pawn in piece_map.values():
        tmp_list.append(pawn)
        # Pawn and Queen will not be on the same board
        # This is in line with the tasks in readme.md
        piece_map[126] = chess.Piece.from_symbol('Q')

    for position in piece_map:
        index = tmp_list.index(piece_map[position])
        tmp_list[index] = None
        int_rep |= (position + 1) << index * 7 + 1

    return int_rep

Die Funktion `to_board(int_rep, piece_list)` wandelt einen Integer und eine Liste von Figuren `piece_list` in einer Spielsituation in ein Board-Objekt. Das geringwertigste Bit, wird in die Information, welcher Spieler am Zug ist, konvertiert. Die verbleibenden Bits werden nacheinander zu einer Zahl gewandelt, welche einem Spielfeld (`chess.Square`) entspricht.

Die bestimmten Informationen über Zug und Piece-Map werden einem neuen Board-Objekt hinzugefügt, welches von der Funktion zurückgegeben wird.

In [ ]:
def to_board(int_rep, piece_list):    
    tmp_list = piece_list.copy()
    pawn = chess.Piece.from_symbol("P")
    tmp_list.append(pawn)

    new_piece_map = {}

    turn = int_rep & 1
    int_rep = int_rep >> 1

    i = 0
    while int_rep > 0:
        pos = (int_rep & 127) - 1
        int_rep = int_rep >> 7
        if pos > 63:
            i += 1
            continue
        new_piece_map[pos] = tmp_list[i]
        i += 1

    res = chess.Board(None)
    res.turn = bool(turn)
    res.set_piece_map(new_piece_map)
    return res


Da für die Berechnungen temporär Listen von ``chess.Board`` Objekten vorliegen wird die Funktion `multiple_to_integer(chess_boards, piece_list)` definiert, welche für eine solche Liste `chess_boards` eine Menge der Integer-Repräsentationen (`integers`) erstellt und zurück gibt.

In [ ]:
def multiple_to_integer(chess_boards, piece_list):
    integers = set()
    for board in chess_boards:
        integers.add(to_integer(board, piece_list))
    return integers
